CAH11-01 Networks and Operating
Systems
Transport Layer Part 2

In [ ]:
import socket

# Create a TCP socket
server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server_socket.bind(('localhost', 65432))
server_socket.listen(1) # Allow 1 pending connection

print("TCP Server is listening...")

while True:
    client_socket, client_address = server_socket.accept()
    print(f"Connected to {client_address}")
    data = client_socket.recv(1024)
    print(f"Received: {data.decode()}")
    # Echo back the data
    client_socket.sendall(b"ACK: " + data)


TCP Server is listening...


Excersise 1

In [ ]:
import socket
client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
client_socket.connect(('localhost', 65432))
message = input("Enter message: ")
client_socket.sendall(message.encode())
response = client_socket.recv(1024)
print(f"Server response: {response.decode()}")
client_socket.close()


In [ ]:
import socket
import datetime

client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

client_socket.connect(('localhost', 65432))

start_time = datetime.datetime.now()

message = input("Enter message: ")
client_socket.sendall(message.encode())

response = client_socket.recv(1024)
print(f"Server response: {response.decode()}")

end_time = datetime.datetime.now()

duration = end_time - start_time
print(f"Time taken to send data: {duration}")

client_socket.close()

Exercise 2

In [ ]:
import socket
import datetime

client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

client_socket.connect(('localhost', 65432))

start_time = datetime.datetime.now()

message = input("Enter message: ")
client_socket.sendall(message.encode())

response = client_socket.recv(1024)
print(f"Server response: {response.decode()}")

end_time = datetime.datetime.now()

duration = end_time - start_time
print(f"Time taken to send data: {duration}")

client_socket.close()

In [ ]:
with open('file_to_send.txt', 'rb') as file:
    data = file.read()
    print(data)

In [ ]:
from cryptography.fernet import Fernet
import socket
import threading

# Generate encryption key
key = Fernet.generate_key()
cipher = Fernet(key)
clients = []
clients_lock = threading.Lock()

def handle_client(client_socket, client_address):
    print(f"New connection from {client_address}")
    clients.append(client_socket)
    client_socket.send(key)
    
    while True:
        try:
            encrypted_message = client_socket.recv(1024)
            if not encrypted_message:
                break
            message = cipher.decrypt(encrypted_message).decode()
            print(f"Decrypted Message from {client_address}: {message}")
            broadcast(encrypted_message, client_socket)
        except ConnectionResetError:
            break
    
    print(f"{client_address} has disconnected")
    clients.remove(client_socket)
    client_socket.close()

def broadcast(message, sender_socket):
    for client in clients:
        if client != sender_socket:
            client.send(message)

server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server_socket.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
server_socket.bind(('localhost', 65432))
server_socket.listen(5)
print("Encrypted Multi-Client TCP Server is listening...")

while True:
    client_socket, client_address = server_socket.accept()
    thread = threading.Thread(target=handle_client, args=(client_socket, client_address))
    thread.start()

: 

Exercise 7

In [1]:
import socket
import threading
from cryptography.fernet import Fernet

# Generate encryption key
key = Fernet.generate_key()
cipher = Fernet(key)
clients = []
clients_lock = threading.Lock()

def handle_client(client_socket, client_address):
    print(f"New connection from {client_address}")
    with clients_lock:
        clients.append(client_socket)
    client_socket.send(key)
    
    try:
        while True:
            encrypted_message = client_socket.recv(1024)
            if not encrypted_message:
                break
            message = cipher.decrypt(encrypted_message).decode()
            print(f"Decrypted Message from {client_address}: {message}")
            broadcast(encrypted_message, client_socket)
    except ConnectionResetError:
        print(f"{client_address} has disconnected")
    finally:
        with clients_lock:
            clients.remove(client_socket)
        client_socket.close()

def broadcast(message, sender_socket):
    with clients_lock:
        for client in clients:
            if client != sender_socket:
                try:
                    client.send(message)
                except ConnectionResetError:
                    clients.remove(client)
                    client.close()

def start_server():
    server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server_socket.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
    server_socket.bind(('localhost', 65432))
    server_socket.listen(5)
    print("Encrypted Multi-Client TCP Server is listening...")
    
    try:
        while True:
            client_socket, client_address = server_socket.accept()
            thread = threading.Thread(target=handle_client, args=(client_socket, client_address))
            thread.start()
    except KeyboardInterrupt:
        print("Server is shutting down...")
    finally:
        server_socket.close()

if __name__ == "__main__":
    start_server()

Encrypted Multi-Client TCP Server is listening...


: 

: 

Exercise 8

In [ ]:
import socket
import threading


def handle_client(client_socket, client_address):
    data = client_socket.recv(1024)
    print(f"Received from {client_address}: {data.decode('utf-8')}")
    client_socket.close()


def start_server():
    server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server_socket.bind(('localhost', 65438))
    server_socket.listen(2)
    print("TCP Server is listening...")

    while True:
        client_socket, client_address = server_socket.accept()
        print(f"Connected to {client_address}")
        thread = threading.Thread(target=handle_client, args=(client_socket, client_address))
        thread.start()


start_server()

TCP Server is listening...
